# Capstone- Week 3 Project

# Part #1     # Part #1     # Part #1     # Part #1     # Part #1     # Part #1     # Part #1

## Install beautifulsoup4
## Install lxml
## Install request

In [1]:
pip install beautifulsoup4 as bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install request

Note: you may need to restart the kernel to use updated packages.


### Import urllib.request to get the url and change to .html file

In [4]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('ISO_3166-1_alpha-1.html', 'w') as fo:
    fo.write(article)

### Load article, turn into soup and get the tables.

In [5]:
from bs4 import BeautifulSoup
article = open('ISO_3166-1_alpha-1.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

In [6]:
import pandas as pd
# Search through the tables for the one with the headings we want.
toronto_df = pd.read_html('ISO_3166-1_alpha-1.html')

headings = ['PostalCode', 'Borough', 'Neighbourhood']
tdf=toronto_df[0]
tdf

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


### Pulled out a dataframe that that only includes "Boroughs" that are "assigned"
### Reset the index

In [7]:
tdf=tdf[tdf.Borough != 'Not assigned']
tdf.reset_index(drop=True, inplace=True)

tdf.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue


### Because there was only one value with "not assigned" in "Neighborhood" 
### replaced it directly with the desired string

In [8]:
tdf['Neighbourhood'].replace("Not assigned", "Queen's Park", inplace=True)
tdf.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Grouped by "Postcode" column
### Separated by ", "
### Reset index

In [9]:
tdf=tdf.groupby("Postcode").agg(lambda x:', '.join(set(x))).reset_index()
tdf

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Richview Gardens, Kingsview Village, St. Phill..."
101,M9V,Etobicoke,"Mount Olive, South Steeles, Thistletown, Silve..."


### Final dataframe- includes 103 rows and 3 columns

In [10]:
tdf.shape

(103, 3)

# Part #2
#### Import dataframe of Lat and Long

In [11]:
import pandas as pd

url="http://cocl.us/Geospatial_data"
c=pd.read_csv(url)
c.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Concat with df from question #1

In [12]:
concat = pd.concat([tdf, c], axis=1)
concat.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


#### Drop Postal Code column (redundant with Postcode)

In [13]:
longlat=concat.drop('Postal Code', axis=1)
longlat.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
longlat.shape

(103, 5)

# Part #3

#### Import needed modules

In [200]:
import numpy as np # library to handle number

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


#### Search for the coordinates of Toronto, Canada

In [201]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canda are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canda are 43.653963, -79.387207.


#### Create map of Toronto using latitude and longitude values

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighbourhood in zip(longlat['Latitude'], longlat['Longitude'], longlat['Borough'], longlat['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
  
map_toronto

#### Verify Foursquare ID and Secret  (credentials)

In [18]:
CLIENT_ID = 'TDDLMIKHZQ4E0VWUBH5OMCP2S21NYTUCKDMP3SJXPYVFVTKO' # your Foursquare ID
CLIENT_SECRET = '4V15SLMDWUEYZ425KZ5HI1VU2HI2HLIAV5APLBKVWGMUMKEO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TDDLMIKHZQ4E0VWUBH5OMCP2S21NYTUCKDMP3SJXPYVFVTKO
CLIENT_SECRET:4V15SLMDWUEYZ425KZ5HI1VU2HI2HLIAV5APLBKVWGMUMKEO


#### Pull out a database of just the information about the Scarborough Borough

In [202]:
scarborough_data = longlat[longlat['Borough'] == 'Scarborough']
scarborough_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


#### Find the geographical coordinates of Scarborough and map the area

In [204]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, Toronto are 43.773077, -79.257774.


In [205]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)
map_scarborough

#### Use Scarborough map to identify Postal Code areas

In [206]:
map_sc = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Postcode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sc)  
    
map_sc

#### Using the data in the first row, find the latitude and longitude

In [24]:
pc_latitude = scarborough_data.loc[0, 'Latitude'] # Postcode latitude value
pc_longitude = scarborough_data.loc[0, 'Longitude'] # Postcode longitude value

pc_name = scarborough_data.loc[0, 'Postcode'] # Postcode name

print('Latitude and longitude values of {} are {}, {}.'.format(pc_name, 
                                                               pc_latitude, 
                                                                pc_longitude))

Latitude and longitude values of M1B are 43.806686299999996, -79.19435340000001.


#### Draw the information from Foursquare for each location based on latitude and longitude by pulling a url

In [208]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    pc_latitude, 
    pc_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TDDLMIKHZQ4E0VWUBH5OMCP2S21NYTUCKDMP3SJXPYVFVTKO&client_secret=4V15SLMDWUEYZ425KZ5HI1VU2HI2HLIAV5APLBKVWGMUMKEO&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [209]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5db7970c8afbe0002d350428'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

#### borrow the get_category_type function from the Foursquare lab.

In [210]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the json and structure it into a *pandas* dataframe.

In [216]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


#### how many venues were retuned by Foursquare?

In [212]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


#### Process venues for other Postcodes in Scarborough

In [213]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each postcode and create a new dataframe called *sc_venues*.

In [217]:
sc_venues = getNearbyVenues(names=scarborough_data['Postcode'],
                      latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )


M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X


#### Check the shape of the data and the general appearance of the dataframe

In [218]:
print(sc_venues.shape)
sc_venues.head()

(88, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


#### Check how many were returned for each Postcode

In [219]:
sc_venues.groupby('Postcode').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M1B,2,2,2,2,2,2
M1C,1,1,1,1,1,1
M1E,7,7,7,7,7,7
M1G,3,3,3,3,3,3
M1H,7,7,7,7,7,7
M1J,2,2,2,2,2,2
M1K,4,4,4,4,4,4
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


#### How many unique categories of venues are there?

In [43]:
print('There are {} uniques categories.'.format(len(sc_venues['Venue Category'].unique())))

There are 52 uniques categories.


#### Change into "dummies" so data can be calculated numerically

In [44]:
# one hot encoding
sc_onehot = pd.get_dummies(sc_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postcode column back to dataframe
sc_onehot['Postcode'] = sc_venues['Postcode'] 

# move Postcode column to the first column
fixed_columns = [sc_onehot.columns[-1]] + list(sc_onehot.columns[:-1])
sc_onehot = sc_onehot[fixed_columns]

sc_onehot.head()

,Postcode,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
sc_onehot.shape

(88, 53)

#### Group rows by neighborhood and take the mean of the frequency of occurrence of each category
#### Reset the index

In [46]:
sc_grouped = sc_onehot.groupby('Postcode').mean().reset_index()
sc_grouped

,Postcode,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,M1B,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.500000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
1,M1C,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
2,M1E,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.142857,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.142857,0.0,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.142857,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
3,M1G,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.666667,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
4,M1H,0.0,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.00,0.142857,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.142857,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.142857,0.000000,0.000000
5,M1J,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.5,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
6,M1K,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.250000,0.00,0.0,0.25,0.5,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
7,M1L,0.0,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.000000
8,M1M,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0

#### Print each neighborhood along with the top 5 most common venues

In [227]:
num_top_venues = 5

for hood in sc_grouped['Postcode']:
    print("----"+hood+"----")
    temp = sc_grouped[sc_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0  Fast Food Restaurant   0.5
1            Print Shop   0.5
2   American Restaurant   0.0
3             Pet Store   0.0
4    Light Rail Station   0.0


----M1C----
                 venue  freq
0                  Bar   1.0
1  American Restaurant   0.0
2             Pharmacy   0.0
3   Light Rail Station   0.0
4               Lounge   0.0


----M1E----
                venue  freq
0         Pizza Place  0.14
1        Intersection  0.14
2      Medical Center  0.14
3      Breakfast Spot  0.14
4  Mexican Restaurant  0.14


----M1G----
                venue  freq
0         Coffee Shop  0.67
1   Korean Restaurant  0.33
2  Light Rail Station  0.00
3              Lounge  0.00
4      Medical Center  0.00


----M1H----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2     Thai Restaurant  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----M1J----
                 venue  freq
0           Playground   0.5
1 

#### Put into a *pandas* dataframe

In [228]:
#First, write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each Post Code.

In [229]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['Postcode'] = sc_grouped['Postcode']

for ind in np.arange(sc_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sc_grouped.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Intersection,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
3,M1G,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment


#### Run *k*-means to cluster the neighborhood into clusters.
*** This data doesn't cluster very well.  I tried different amounts of clusters and there are always several that only have 1 data set and one or two that have a large number.  It doesn't seem to be a helpful way to sort this particular data.

In [230]:
# set number of cluster
kclusters = 5

sc_grouped_clustering = sc_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 3, 1, 4, 1, 1, 1, 1], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each Post Code.

In [231]:
# add clustering labels
pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sc_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sc_merged = sc_merged.join(pc_venues_sorted.set_index('Postcode'), on='Postcode')
sc_merged.head(18) 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711,1.0,Intersection,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4.0,Convenience Store,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029,1.0,Discount Store,Department Store,Coffee Shop,Vietnamese Restaurant,Thrift / Vintage Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Park,Coffee Shop,General Entertainment
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476,1.0,American Restaurant,Motel,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Map the clustered data
*** Note that the last row had NaN data- a line was added to not include this data

In [232]:
import math

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sc_merged['Latitude'], sc_merged['Longitude'], sc_merged['Postcode'], sc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    if math.isnan(cluster):
        continue
    color_i = math.floor(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[color_i],
        fill=True,
        fill_color=rainbow[color_i],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

### Examine each cluster of information

In [233]:
sc_merged.loc[sc_merged['Cluster Labels'] == 0, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,0.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [189]:
sc_merged.loc[sc_merged['Cluster Labels'] == 1, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,1.0,Intersection,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
4,M1H,1.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Grocery Store,General Entertainment
7,M1L,1.0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Park,Coffee Shop,General Entertainment
10,M1P,1.0,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Bar,Breakfast Spot,Grocery Store,General Entertainment
11,M1R,1.0,Auto Garage,Smoke Shop,Bakery,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Breakfast Spot,Vietnamese Restaurant,Discount Store,College Stadium
13,M1T,1.0,Pizza Place,Breakfast Spot,Bank,Noodle House,Pharmacy,Rental Car Location,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Fried Chicken Joint
15,M1W,1.0,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Coffee Shop,Thrift / Vintage Store,Electronics Store,Sandwich Place,Breakfast Spot,Pizza Place,Pharmacy


In [190]:
sc_merged.loc[sc_merged['Cluster Labels'] == 2, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M1M,2.0,American Restaurant,Motel,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [191]:
sc_merged.loc[sc_merged['Cluster Labels'] == 3, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M1J,3.0,Convenience Store,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [192]:
sc_merged.loc[sc_merged['Cluster Labels'] == 4, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M1V,4.0,Bakery,Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [193]:
sc_merged.loc[sc_merged['Cluster Labels'] == 5, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M1G,5.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [194]:
sc_merged.loc[sc_merged['Cluster Labels'] == 5, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M1G,5.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [195]:
sc_merged.loc[sc_merged['Cluster Labels'] == 6, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1K,6.0,Discount Store,Department Store,Coffee Shop,Vietnamese Restaurant,Thrift / Vintage Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [196]:
sc_merged.loc[sc_merged['Cluster Labels'] == 7, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,7.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [197]:
sc_merged.loc[sc_merged['Cluster Labels'] == 8, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M1N,8.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [198]:
sc_merged.loc[sc_merged['Cluster Labels'] == 9, sc_merged.columns[[0] + list(range(5, sc_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1S,9.0,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
